In [1]:
import pathlib
import tensorflow as tf

train_dir = pathlib.Path('DRAM_train/')
test_dir = pathlib.Path('DRAM_test/')

m = 224
batch = 16

In [2]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=(m, m),
    batch_size=batch
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(m, m),
    batch_size=batch
)
num_classes = 4

Found 5677 files belonging to 4 classes.
Found 583 files belonging to 4 classes.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.applications.ResNet50(include_top=False, pooling='avg', weights=None, input_shape=(m, m, 3)),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.layers[1].trainable = False

model.compile(
  optimizer=tf.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=11
)

Epoch 1/11
355/355 [==============================] - 236s 656ms/step - loss: 1.3607 - accuracy: 0.2877 - val_loss: 1.4100 - val_accuracy: 0.2419
Epoch 2/11
355/355 [==============================] - 227s 638ms/step - loss: 1.3458 - accuracy: 0.3333 - val_loss: 1.4042 - val_accuracy: 0.2333
Epoch 3/11
355/355 [==============================] - 188s 528ms/step - loss: 1.3336 - accuracy: 0.3562 - val_loss: 1.3994 - val_accuracy: 0.2470
Epoch 4/11
355/355 [==============================] - 228s 643ms/step - loss: 1.3228 - accuracy: 0.3729 - val_loss: 1.3955 - val_accuracy: 0.2384
Epoch 5/11
355/355 [==============================] - 192s 540ms/step - loss: 1.3134 - accuracy: 0.3851 - val_loss: 1.3924 - val_accuracy: 0.2401
Epoch 6/11
355/355 [==============================] - 194s 546ms/step - loss: 1.3050 - accuracy: 0.3932 - val_loss: 1.3898 - val_accuracy: 0.2504
Epoch 7/11
355/355 [==============================] - 191s 538ms/step - loss: 1.2976 - accuracy: 0.4013 - val_loss: 1.3878 -

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 4)                 8196      
                                                                 
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________
